# Part 2, 3, and 4 (Python and Discussion)

My code for this can be found here as well on this public Github: https://github.com/Sam-v6/mece-6397-doe/tree/main/project1

## Importing Modules
For starters I import pandas, numpy, and combonations which helps generate all possible combos to examine interactions of different factors.

In [2]:
"""
Purpose: Project 1 - Driver script for factorial design and main effects calculation
Author: Syam Evani
"""

# Standard imports
import os

# Additional imports
import pandas as pd
import numpy as np
from itertools import combinations

# Local imports
# None

## Creating the design

To create the design I define 2 levels (a low and high) and 5 factors which matches the JMP simulation that we did in the prior section. Notice that for our factors I defined a low and high value similar to that of the jmp simulation. 

I generate the experiment design, which at this point is just 0 or 1s indicating low or highs. I save the history of each design into the "pattern" to ultimately display that in the final design output, similar to that of the JMP simulation. I then translate these low and high boolean values into the actual minimum or maximum values specific to the factor itself (for example feed rate low is 10 and high is 15).

I then create a fictional response that we call the "percent reacted" using random number generation. 

In [3]:
#------------------------------------------------------------------------
# DOE Design Section
#------------------------------------------------------------------------
# Define factors and their levels
levels = [0, 1]  # Assuming -1 is low, 0 is medium, and +1 is high
factors={
        'Feed Rate': {"Min": 10, "Max": 15},
        'Catalyst':{"Min": 1, "Max": 2},
        'Stir Rate':{"Min": 100, "Max": 120},
        'Temperature':{"Min": 140, "Max": 180}, 
        'Concentration':{"Min": 3, "Max": 6}
        }

# Generate full factorial design
experiment_design = pd.DataFrame(np.array(np.meshgrid(levels, levels, levels, levels, levels)).T.reshape(-1, 5), columns=factors.keys())

# Capture what pattern was generated
pattern_list = []
for i in range(0,32):
    pattern = []
    for factor in experiment_design.columns:
        pattern.append(experiment_design[factor][i])
    pattern_list.append(pattern)

# Map the levels to their corresponding min and max values
for factor in experiment_design.columns:
    for i in range(0,32):
        if experiment_design[factor][i] == 0:
            experiment_design[factor][i] = factors[factor]["Min"]
        else:
            experiment_design[factor][i] = factors[factor]["Max"]

# Add the pattern and percent reacted to the design
experiment_design['Pattern'] = pattern_list

# Display the design with percent reacted
np.random.seed(42)  # For reproducibility
experiment_design['Percent Reacted'] = np.random.rand(len(experiment_design))*100

## Main Effects Calculation

Similar to the main effects calculation in the provided code, I simply group the factors and evalute the mean for percent reacted to see how each factor plays into impacting overall percent reacted.

In [4]:
#------------------------------------------------------------------------
# DOE Main Effects Calculation
#------------------------------------------------------------------------
main_effects = {}
for factor in factors.keys():
    main_effects[factor] = experiment_design.groupby(factor)['Percent Reacted'].mean()

## Interaction Effects Calculation

To calculate the interactions effects I generate all combonations of the factors (from 2 factors out to 5 factors). I then can similarly group these combonations of factors and evalute the mean of percent reacted to see the interaction effects.

In [5]:
#------------------------------------------------------------------------
## DOE Interaction Effects Calculation
#------------------------------------------------------------------------
# Calculate interaction effects
interaction_effects = {}
factor_names = list(factors.keys())
for r in range(2, len(factor_names) + 1):
    for combo in combinations(factor_names, r):
        interaction_term = ' x '.join(combo)
        interaction_effects[interaction_term] = experiment_design.groupby(list(combo))['Percent Reacted'].mean().unstack()

## Creating contrast and printing

Similar to the JMP simulation, I want to evaluate the "contrast", which effectively is which direction does it drive the mean of percent reacted. This can be determined by determining the difference between the mean associated with the low and high level of either the main factor or combonation of factors.

I then can sort the contrast from most impactful (positively impact percent reacted) to least impactful and output/print this text file to see the full rankings to determine which factors or combonations will be best to increase percent reacted.

In [6]:
#------------------------------------------------------------------------
# Generate Contrast Output
#------------------------------------------------------------------------
contrast_output = pd.DataFrame(columns=['Factor/Interaction', 'Low Level Mean', 'High Level Mean', 'Effect'])
contrast_rows = []

# Calculate contrast for main effects
for factor in factors.keys():
    low_level_mean = main_effects[factor].iloc[0]
    high_level_mean = main_effects[factor].iloc[1]
    effect = high_level_mean - low_level_mean
    contrast_rows.append({
        'Factor/Interaction': factor,
        'Low Level Mean': low_level_mean,
        'High Level Mean': high_level_mean,
        'Effect': effect
    })

# Calculate contrast for interaction effects
for interaction_term, interaction_data in interaction_effects.items():
    for idx, (level, row) in enumerate(interaction_data.iterrows()):
        low_level_mean = row.iloc[0]
        high_level_mean = row.iloc[1]
        effect = high_level_mean - low_level_mean
        contrast_rows.append({
            'Factor/Interaction': f'{interaction_term} (Level {level})',
            'Low Level Mean': low_level_mean,
            'High Level Mean': high_level_mean,
            'Effect': effect
        })

# Save contrast and sort from highest to lowest
contrast_output = pd.DataFrame(contrast_rows)
contrast_output = contrast_output.sort_values(by='Effect', ascending=False)

In [7]:
#------------------------------------------------------------------------
# Print design, main effects contrast, and overall constrast rankings
#------------------------------------------------------------------------
# Print design
print("Full Factorial Design and Percent Reacted:")
print(experiment_design)

# Print main effects constrast
print("\nMain Effects:")
for factor, effects in main_effects.items():
    print(f"\n{factor}:")
    print(effects)

# Save contrast to text file for interactions
with open(os.path.join(os.getenv('USERPROFILE'),"repos","mece-6397-doe","project1","output","contrast_output.txt"), "w") as file:
    file.write(contrast_output.to_string(index=False))

# Print overall rankings
with open(os.path.join(os.getenv('USERPROFILE'),"repos","mece-6397-doe","project1","output","contrast_output.txt")) as file:
    contents = file.read()

print(contents)

Full Factorial Design and Percent Reacted:
    Feed Rate  Catalyst  Stir Rate  Temperature  Concentration  \
0          10         1        100          140              3   
1          10         2        100          140              3   
2          15         1        100          140              3   
3          15         2        100          140              3   
4          10         1        120          140              3   
5          10         2        120          140              3   
6          15         1        120          140              3   
7          15         2        120          140              3   
8          10         1        100          180              3   
9          10         2        100          180              3   
10         15         1        100          180              3   
11         15         2        100          180              3   
12         10         1        120          180              3   
13         10         2        12

## Conclusion

With our randomly generated responses (percent reacted) and with our random seed (42), we find that increasing the Catalyst level is the most impactful, single factor that can improve percent reacted with a 6% contribution. An overall grading of factors is as follows in the following form (Factor, Percented Reacted Change)

1) Catalyst (+6%)
2) Temperature (+1.4%)
3) Feed Rate (-6%)
4) Concentration (-7.9%)
5) Stir Rate (-18.7%)

Notice that increasing the level of Feed Rate, Concentration, and Stir Rate negatively impacts percent reacted if we are examining these factors indvidually. However, the individual examinination in itself doesn't fully capture coupled impacts, therefore investigating combonations of factors percented reacted change is more valuable. I generated 121 combonations of different factors and ultimately found some combonations of the 5 factors yield the best mean change in percent reacted. For example the top 2 combonations include:
                     
1) Feed Rate x Catalyst x Stir Rate x Temperature x Concentration (Level (10, 1, 120, 140)) which has a Percent Reacted Change of +45.6%
2) Feed Rate x Catalyst x Stir Rate x Temperature x Concentration (Level (15, 1, 120, 180)) which has a Percented Reacted Change of +42.6%

In esssence, if we have those factors with their accompanying factors and then increase the concentration, this will dramatically and postiviely impact our overall chemcial reaction percent reacted. Following these combos of 5 factors, the next most impactful combos are 4 factors that follow a similar trend. Commenting on some pair combonations, the top two paired combonations positively impacting percent reacted are:

1) Feed Rate x Catalyst (Level 15) which has a Percent Reacted Change of +18%
2) Feed Rate x Temperature (Level 10) which has a Percent Reacted Change of +12.7%
3) Catalyst x Concentration (Level 1) which has a Percent Reacted Change of +6.7%


As evidenced, each of these factors has unique, coupled impacts when combined and it's important to understand the nuance associated with each of these. If I was an engineer on this experiment, I would keep in the back of my mind the overall net increase that increasing the catalyst level can bring, however, I would specifically pay attention to the coupled impacts with multiple factors and atttempt to understand the underlying phyiscal reasoning to better increase the overall net, percent reacted for my chemical reactions.